In [1]:
import jax, jaxlib, os
print("jax =", jax.__version__)
print("jaxlib =", jaxlib.__version__)
print("backend =", jax.default_backend())
print("platform_version =", jax.lib.xla_bridge.get_backend().platform_version)  # 看 CUDA/驱动信息
print("devices =", jax.devices())

jax = 0.7.1
jaxlib = 0.7.1
backend = gpu
platform_version = PJRT C API
cuda 12090
devices = [CudaDevice(id=0)]


/tmp/ipykernel_206834/702583580.py:5: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated and will be removed in JAX v0.8.0; use jax.extend.backend.get_backend.
  print("platform_version =", jax.lib.xla_bridge.get_backend().platform_version)  # 看 CUDA/驱动信息


In [2]:
# @title Import packages for plotting and creating graphics
import json
import itertools
import time
from typing import Callable, List, NamedTuple, Optional, Union
import numpy as np

# # Graphics and plotting.
# print("Installing mediapy:")
# !command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
# !pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [3]:
# @title Import MuJoCo, MJX, and Brax
from datetime import datetime
import functools
import os
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.base import State as PipelineState
from brax.envs.base import Env, PipelineEnv, State
from brax.io import html, mjcf, model
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import networks as sac_networks
from brax.training.agents.sac import train as sac
from etils import epath
from flax import struct
from flax.training import orbax_utils
from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
import numpy as np
from orbax import checkpoint as ocp

In [4]:
#@title Import The Playground

from mujoco_playground import wrapper
from mujoco_playground import registry

In [5]:
# Mujoco Playground 自带的训练环境
registry.locomotion.ALL_ENVS

('ApolloJoystickFlatTerrain',
 'BarkourJoystick',
 'BerkeleyHumanoidJoystickFlatTerrain',
 'BerkeleyHumanoidJoystickRoughTerrain',
 'G1JoystickFlatTerrain',
 'G1JoystickRoughTerrain',
 'Go1JoystickFlatTerrain',
 'Go1JoystickRoughTerrain',
 'Go1Getup',
 'Go1Handstand',
 'Go1Footstand',
 'H1InplaceGaitTracking',
 'H1JoystickGaitTracking',
 'Op3Joystick',
 'SpotFlatTerrainJoystick',
 'SpotGetup',
 'SpotJoystickGaitTracking',
 'T1JoystickFlatTerrain',
 'T1JoystickRoughTerrain')

In [6]:
# 选择对应训练环境
env_name = 'Go1JoystickFlatTerrain'
env = registry.load(env_name)
env_cfg = registry.get_default_config(env_name)

# 查看环境参数
env_cfg

Kd: 0.5
Kp: 35.0
action_repeat: 1
action_scale: 0.5
command_config:
  a:
  - 1.5
  - 0.8
  - 1.2
  b:
  - 0.9
  - 0.25
  - 0.5
ctrl_dt: 0.02
episode_length: 1000
history_len: 1
noise_config:
  level: 1.0
  scales:
    gravity: 0.05
    gyro: 0.2
    joint_pos: 0.03
    joint_vel: 1.5
    linvel: 0.1
pert_config:
  enable: false
  kick_durations:
  - 0.05
  - 0.2
  kick_wait_times:
  - 1.0
  - 3.0
  velocity_kick:
  - 0.0
  - 3.0
reward_config:
  max_foot_height: 0.1
  scales:
    action_rate: -0.01
    ang_vel_xy: -0.05
    dof_pos_limits: -1.0
    energy: -0.001
    feet_air_time: 0.1
    feet_clearance: -2.0
    feet_height: -0.2
    feet_slip: -0.1
    lin_vel_z: -0.5
    orientation: -5.0
    pose: 0.5
    stand_still: -1.0
    termination: -1.0
    torques: -0.0002
    tracking_ang_vel: 0.5
    tracking_lin_vel: 1.0
  tracking_sigma: 0.25
sim_dt: 0.004
soft_joint_pos_limit_factor: 0.95

In [7]:
# 加载 PPO 训练参数
from mujoco_playground.config import locomotion_params
ppo_params = locomotion_params.brax_ppo_config(env_name)
ppo_params

action_repeat: 1
batch_size: 256
discounting: 0.97
entropy_cost: 0.01
episode_length: 1000
learning_rate: 0.0003
max_grad_norm: 1.0
network_factory:
  policy_hidden_layer_sizes: &id001 !!python/tuple
  - 512
  - 256
  - 128
  policy_obs_key: state
  value_hidden_layer_sizes: *id001
  value_obs_key: privileged_state
normalize_observations: true
num_envs: 8192
num_evals: 10
num_minibatches: 32
num_resets_per_eval: 1
num_timesteps: 200000000
num_updates_per_batch: 4
reward_scaling: 1.0
unroll_length: 20

In [8]:
# 添加随机化 Domain Randomization
registry.get_domain_randomizer(env_name)

<function mujoco_playground._src.locomotion.go1.randomize.domain_randomize(model: mujoco.mjx._src.types.Model, rng: jax.Array)>

## 可视化

In [9]:
# # Enable perturbation in the eval env.
# env_cfg = registry.get_default_config(env_name)
# env_cfg.pert_config.enable = True
# env_cfg.pert_config.velocity_kick = [3.0, 6.0]
# env_cfg.pert_config.kick_wait_times = [5.0, 15.0]
# env_cfg.command_config.a = [1.5, 0.8, 2*jp.pi]
# eval_env = registry.load(env_name, config=env_cfg)
# velocity_kick_range = [0.0, 0.0]  # Disable velocity kick.
# kick_duration_range = [0.05, 0.2]

# jit_reset = jax.jit(eval_env.reset)
# jit_step = jax.jit(eval_env.step)

In [ ]:
#@title 仅可视化机器人 (无策略)
# 这个版本只展示机器人的默认行为，不使用任何训练的策略

# 重新加载环境，不需要策略推理
simple_env = registry.load(env_name, config=env_cfg)
simple_jit_reset = jax.jit(simple_env.reset)
simple_jit_step = jax.jit(simple_env.step)

# 设置可视化参数
x_vel = 0.0  #@param {type: "number"}
y_vel = 0.0  #@param {type: "number"}
yaw_vel = 0.0  #@param {type: "number"}

rng = jax.random.PRNGKey(42)
rollout = []
command = jp.array([x_vel, y_vel, yaw_vel])

# 重置环境
state = simple_jit_reset(rng)
state.info["command"] = command

# 进行rollout，但使用零动作或简单的PD控制
rollout_steps = 200  # 减少步数以便快速可视化

for i in range(rollout_steps):
    # 使用零动作 - 机器人会自然下落并展示物理效果
    ctrl = jp.zeros(simple_env.action_size)
    
    # 或者使用简单的位置控制来保持站立姿态 (可选)
    # 如果你想让机器人保持站立，可以取消下面几行的注释
    # default_pose = jp.array([0.0, 0.8, -1.6, 0.0, 0.8, -1.6,  # 前腿
    #                         0.0, 0.8, -1.6, 0.0, 0.8, -1.6])   # 后腿  
    # ctrl = default_pose
    
    state = simple_jit_step(state, ctrl)
    state.info["command"] = command
    rollout.append(state)

# 渲染设置
render_every = 2  # 每4帧渲染一次以提高性能
fps = 1.0 / simple_env.dt / render_every
traj = rollout[::render_every]

# 设置场景选项
scene_option = mujoco.MjvOption()
scene_option.geomgroup[2] = True
scene_option.geomgroup[3] = False
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
scene_option.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = False

# 渲染视频
frames = simple_env.render(
    traj,
    camera="“track”",  # 可以尝试 "side", "back", “track” 等不同视角
    scene_option=scene_option,
    width=600,
    height=400,
)

print(f"生成了 {len(frames)} 帧，帧率: {fps:.1f} FPS")
media.show_video(frames, fps=fps, loop=True)

100%|██████████| 100/100 [00:00<00:00, 108.12it/s]


生成了 100 帧，帧率: 25.0 FPS


This browser does not support the video tag.
